In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
import os

# 경로 설정
current_dir = os.getcwd()
project_root = os.path.dirname(os.path.dirname(current_dir))
DATA_DIR = os.path.join(project_root, 'data', 'modeling')

# 데이터 로딩
grid_cluster = pd.read_csv(os.path.join(DATA_DIR, 'grid_with_cluster.csv'))

# cluster 컬럼이 결측이 아닌(클러스터링된) 격자만 남김
grid_df = grid_df[grid_df['cluster'].notnull()]

# 서울시 범위 내 격자만 필터링
def in_seoul(lat, lon):
    return 37.413294 <= lat <= 37.715133 and 126.734086 <= lon <= 127.269311
grid_df = grid_df[grid_df.apply(lambda row: in_seoul(row['center_lat'], row['center_lon']), axis=1)]

# 클러스터별 색상
num_clusters = grid_df['cluster'].nunique()
cmap = plt.get_cmap('tab20', num_clusters)
def rgba_to_hex(rgba):
    return '#{:02x}{:02x}{:02x}'.format(
        int(rgba[0]*255), int(rgba[1]*255), int(rgba[2]*255)
    )
cluster_colors = [rgba_to_hex(cmap(i)) for i in range(num_clusters)]

# 지도 생성
center_lat, center_lon = 37.5665, 126.9780
m = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles='CartoDB positron')

# 서울 경계선 geojson 오버레이
boundary_path = 'data/seoul_boundary.geojson'
if os.path.exists(boundary_path):
    folium.GeoJson(
        boundary_path,
        name='Seoul Boundary',
        style_function=lambda x: {
            'color': 'black',
            'weight': 2,
            'fill': False
        }
    ).add_to(m)

# 클러스터링된 격자에만 색상 채우기
for _, row in grid_df.iterrows():
    cluster_id = int(row['cluster'])
    color = cluster_colors[cluster_id % len(cluster_colors)]
    bounds = [
        [row['min_lat'], row['min_lon']],
        [row['max_lat'], row['max_lon']]
    ]
    folium.Rectangle(
        bounds=bounds,
        color=color,
        weight=0.5,
        opacity=0.2,
        fill=True,
        fill_color=color,
        fill_opacity=0.4,
        popup=folium.Popup(
            f"<b>Grid:</b> {row['grid_id']}<br>"
            f"<b>Cluster:</b> {cluster_id}<br>"
            f"<b>수요:</b> {row['demand_score']:.1f}<br>"
            f"<b>공급:</b> {row['supply_score']:.1f}",
            max_width=250
        )
    ).add_to(m)

folium.LayerControl(position='topright', collapsed=False).add_to(m)

m

FileNotFoundError: [Errno 2] No such file or directory: 'd:\\projects\\2025-1\\빅데이터프로그래밍\\data\\modeling\\grid_with_cluster.csv'